## Задание 10. — Сеточные методы для уравнений в частных производных

* Реализовать решение уравнения теплопроводности по двум схемам: одной из неявных и явной.
* Посмотреть на поведение решения по явной схеме при несоблюдении условий устойчивости.
* Результаты выводить либо графически (поверхность), либо численно (матрицу значений).

## Решение

In [1]:
import numpy as np
import pandas
from scipy.misc import derivative

import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt

Сперва зададим функцию, которая строит необходимое уравнение по некоторому заданному решению:

In [2]:
def cond_from_sol(equation, space_border, kappa):
    '''
        Функция, строящая уравненение теплопроводности по некоторому решению

        Parameters
        ----------
        equation : function
            функция, являющаяся решением уравнения теплопроводности
        space_border : int
            граница по пространственной переменной
        kappa : int
            значение коэффициента каппа
        
        Returns
        ----------
        de : function
            уравнение теплопроводности
        m : function
            функция-коэффициент мю
        m1 : function
            функция-коэффициент мю_1
        m2 : function
            функция-коэффициент мю_2
    '''
    deq_dt  = lambda x0, i0: derivative(lambda t: equation(x0, t), i0)
    deq2_dx = lambda x0, i0: derivative(lambda x: equation(x, i0), x0, n=2)
    
    m  = lambda x: equation(x, 0)
    m1 = lambda i: equation(0, i)
    m2 = lambda i: equation(space_border, i)
    
    de = lambda x, t: deq_dt(x, t) - kappa*deq2_dx(x, t)
    
    return de, m, m1, m2

Далее зададим реализацию решения по явной схеме:

In [3]:
def explicit_scheme(de, m, m1, m2, borders, partition, kappa):
    '''
        Решение уравнения теплопроводности по явной схеме

        Parameters
        ----------
        de : function
            уравнение теплопроводности
        m : function
            функция-коэффициент мю
        m1 : function
            функция-коэффициент мю_1
        m2 : function
            функция-коэффициент мю_2    
        borders : list
            массив с границами по пространственной и временной переменной
        partition : list
            массив с количеством разбиений сетки по пространственной и временной переменной
        kappa : int
            значение коэффициента каппа
        
        Returns
        ----------
        grid_spase : numpy.array
            сетка по пространственной переменной
        grid_time : numpy.array
            сетка по временной переменной
        grid_sol : numpy.array
            сетка с решением 
    '''
    # достаём начальные параметры
    space, time = borders
    N, M = partition
    h = space / N
    t = time  / M
    
    # задаём сетки по переменным
    grid_space = np.array([i*h for i in range(0, N + 1)])
    grid_time  = np.array([i*t for i in range(0, M + 1)])
    
    # инициализируем сетку решения
    grid_sol = []
    for i in range(N + 1):
        grid_sol.append([0. for _ in range(0, M + 1)])
        grid_sol[i][0] = m(grid_space[i])
        
    # заполняем сетку по явной схеме
    for l in range(1, M + 1):
        for i in range(1, N):
            des = de(grid_space[i], grid_time[l - 1])
            diff = grid_sol[i - 1][l - 1] - 2*grid_sol[i][l - 1] + grid_sol[i + 1][l - 1]
            grid_sol[i][l] = grid_sol[i][l - 1] + t*(des + (diff*kappa) / h ** 2)
        grid_sol[0][l] = m1(grid_time[l])
        grid_sol[N][l] = m2(grid_time[l])
    grid_sol = np.array(grid_sol)
    
    return grid_space, grid_time, grid_sol

И функцию для тестирования:

In [4]:
def explicit_testing(eq, borders, partition, kappa, need_plot=True):
    '''
        Функция, тестирующая явную схему

        Parameters
        ----------
        eq : function
            некоторое решение
        borders : list
            массив с границами по пространственной и временной переменной
        partition : list
            массив с количеством разбиений сетки по пространственной и временной переменной
        kappa : int
            значение коэффициента каппа
        need_plot: bool
            необходимо ли рисовать график
    '''
    de, m, m1, m2 = cond_from_sol(eq, borders[0], kappa)
    grid_space, grid_time, grid_sol = explicit_scheme(de, m, m1, m2, borders, partition, kappa)
    
    if need_plot:
        fig = go.Figure(data=[go.Surface(x=grid_space, y=grid_time, z=grid_sol)])
        fig.show()
        
    # Проверка устойчивости
    if 2*kappa*(borders[1] / partition[1]) <= (borders[0] / partition[0]) ** 2:
        print('Условие устойчивости выполнено!')
    else:
        print('Условие устойчивости не выполнено!')

И рассмотрим на кусочке эллиптического параболоида:

In [10]:
explicit_testing(
    eq = lambda x, t: (t ** 2) + (x ** 2),
    borders = [20, 20], 
    partition = [100, 100], 
    kappa = 0.01
)

Условие устойчивости выполнено!


А если условие нарушить...

In [11]:
explicit_testing(
    eq = lambda x, t: (t ** 2) + (x ** 2),
    borders = [20, 20], 
    partition = [100, 100], 
    kappa = 1
)

Условие устойчивости не выполнено!


Зададим реализацию решения по неявной схеме:

In [5]:
def implicit_scheme(de, m, m1, m2, borders, partition, kappa):
    '''
        Решение уравнения теплопроводности по явной схеме

        Parameters
        ----------
        de : function
            уравнение теплопроводности
        m : function
            функция-коэффициент мю
        m1 : function
            функция-коэффициент мю_1
        m2 : function
            функция-коэффициент мю_2    
        borders : list
            массив с границами по пространственной и временной переменной
        partition : list
            массив с количеством разбиений сетки по пространственной и временной переменной
        kappa : int
            значение коэффициента каппа
        
        Returns
        ----------
        grid_spase : numpy.array
            сетка по пространственной переменной
        grid_time : numpy.array
            сетка по временной переменной
        grid_sol : numpy.array
            сетка с решением 
    '''
    # достаём начальные параметры
    space, time = borders
    N, M = partition
    h = space / N
    t = time  / M
    
    # задаём сетки по переменным
    grid_space = np.array([i*h for i in range(0, N + 1)])
    grid_time = np.array([i*t for i in range(0, M + 1)])
    
    # векторы из левой части слау
    A, B = np.zeros(N + 1), np.ones(N + 1)
    C, G = np.zeros(N + 1), np.zeros(N + 1)
    
    # инициализируем сетку решения
    grid_sol = [[0. for _ in range(0, M + 1)] for _ in range(0, N + 1)]
    for i in range(0, N + 1):
        grid_sol[i][0] = m(grid_space[i])
    
    # заполняем сетку по неявной схеме
    for l in range(1, M + 1):
        G[0] = m1(grid_time[l])
        G[N] = m2(grid_time[l])
        for i in range(1, N):
            A[i], B[i] = kappa / (h ** 2), (-2*kappa / (h ** 2)) - (1 / t)
            C[i], G[i] = kappa / (h ** 2), -(grid_sol[i][l - 1] / t) - de(grid_space[i], grid_time[l])
        
        # (метод прогонки для решения слау c трёхдиагональной матрицей)
        _s = [0. if i != 0 else -C[0] / B[0] for i in range(0, N + 1)]
        _t = [0. if i != 0 else  G[0] / B[0] for i in range(0, N + 1)]
        for i in range(1, N + 1):
            _s[i] = - (C[i] / (B[i] + _s[i - 1]*A[i]))
            _t[i] = (G[i] - _t[i - 1]*A[i]) / (B[i] + _s[i - 1]*A[i])
        
        grid_sol[N][l] = _t[N]
        for i in range(0, N):
            grid_sol[N - 1 - i][l] = _t[N - 1 - i] + grid_sol[N - i][l]*_s[N - 1 - i]
    
    grid_sol = np.array(grid_sol)
    return grid_space, grid_time, grid_sol

И функцию для её тестирования:

In [6]:
def implicit_testing(eq, borders, partition, kappa, need_plot=True):
    '''
        Функция, тестирующая явную схему

        Parameters
        ----------
        eq : function
            некоторое решение
        borders : list
            массив с границами по пространственной и временной переменной
        partition : list
            массив с количеством разбиений сетки по пространственной и временной переменной
        kappa : int
            значение коэффициента каппа
        need_plot: bool
            необходимо ли рисовать график
    '''
    de, m, m1, m2 = cond_from_sol(eq, borders[0], kappa)
    grid_space, grid_time, grid_sol = implicit_scheme(de, m, m1, m2, borders, partition, kappa)
    
    if need_plot:
        fig = go.Figure(data=[go.Surface(x=grid_space, y=grid_time, z=grid_sol)])
        fig.show()

И также рассмотрим на части эллиптического параболоида:

In [8]:
implicit_testing(
    eq = lambda x, t: (t ** 2) + (x ** 2),
    borders = [20, 20], 
    partition = [100, 100], 
    kappa = 0.01
)

А теперь поменяем параметр $\kappa$...

In [9]:
implicit_testing(
    eq = lambda x, t: (t ** 2) + (x ** 2),
    borders = [20, 20], 
    partition = [100, 100], 
    kappa = 1
)